In [1]:
# # # local
project_directory = "../"


# # # # colab
# from google.colab import drive
# drive.mount('/content/drive')
# project_directory = "/content/drive/MyDrive/colab_working_directory/diversity-enforced-ensembles/"
# !pip install cached-property

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

# allow import of decompose locally
import sys
sys.path.append(project_directory + 'src/')

from decompose import SquaredLoss
import bvdlib

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.func import stack_module_state
from torch.func import functional_call
from torch import vmap
import copy

from bvdlib.trial import Trial
from numpy.random import choice
from numpy.random import seed
from numpy import array
import numpy as np
import pickle

In [3]:
save_path = project_directory + "experiments/results/NCL_Regression_CaliforniaHousing.pkl"

In [4]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

x = housing['data']
y = housing['target']

print("x shape", x.shape)
print("y shape", y.shape)

x shape (20640, 8)
y shape (20640,)


In [5]:
n_trials = 100
trial_space = np.arange(1,21) # Test estimators from 1 to 20
data_percentage_training = int(0.8 * len(y))
num_training = int(0.8 * data_percentage_training)

In [6]:
def init_layer(layer, generator):
    torch.nn.init.xavier_uniform_(layer.weight, generator=generator)
    layer.bias.data.fill_(0.01)

class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, generator):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        init_layer(self.fc1, generator)

        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        init_layer(self.fc2, generator)

        self.fc3 = nn.Linear(hidden_dim, output_dim)
        init_layer(self.fc3, generator)

    def forward(self, x):

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)

        return x

In [7]:
class trial_dataset(torch.utils.data.Dataset):
  def __init__(self, x, y, device):
    self.x = torch.tensor(x).type(torch.float).to(device)
    self.y = torch.tensor(y).type(torch.float).to(device)

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [8]:
experiment_seed = 0
np.random.seed(experiment_seed)
rng = np.random.default_rng()
shuffled_indices = rng.permutation(len(y))

train_indices = shuffled_indices[:data_percentage_training]
test_indices = shuffled_indices[data_percentage_training:]
train_data = x[train_indices, :]
train_labels = y[train_indices]
test_data = x[test_indices, :]
test_labels = y[test_indices]

In [9]:
def forward_through_metamodel(params, buffers, x):
        return functional_call(meta_model, (params, buffers), (x,))

def torch_MSE_combiner(ensemble_output):
    return torch.mean(ensemble_output, axis=0)

combiner_rule = torch_MSE_combiner

def ens_forward(input, ensemble):

  params, buffers = stack_module_state(nn.ModuleList(ensemble))

  member_output = vmap(forward_through_metamodel)(params, buffers, input.repeat(len(ensemble), 1, 1))

  ensemble_output = combiner_rule(member_output)
  return ensemble_output, member_output

In [10]:
class ResultsObject(object):
    """
    Results from BVDExperiment are stored in ResultsObject instances.

    Parameters
    ----------
    parameter_name : str
        name of parameter that is varied over the course of the experiment
    parameter_values : list
        list of values that the varied parameter takes
    loss_func : str
        name of the loss function used for the decompose
    n_test_splits : int
        Number of separate folds unseen data is split into. Default is 2, with the first being the train split and the
        second being test

    Attributes
    ----------
    ensemble_risk : ndarray of shape (n_parameter_values, n_test_splits)
        The risk of the ensemble for each parameter value and test split
    ensemble_bias: ndarray of shape (n_parameter_values, n_test_splits)
        The biasof the ensemble for each paramter value and test split
    ensemble_variance: ndarray of shape (n_parameter_values, n_test_splits)
        The varianceof the ensemble for each parameter value and test split
    average_bias : ndarray of shape (n_parameter_values, n_test_splits)
        The average bias of the ensemble members for each parmater value and test split
    average_variance : ndarray of shape (n_parameter_values, n_test_splits)
        The average variance of the ensemble members for each parmater value and test split
    diversity : ndarray of shape (n_parameter_values, n_test_splits)
        The diversity for each parameter value and test split
    test_error : ndarray of shape (n_parameter_values, n_test_splits)
        The test error of the ensemble for each parameter value and test split
    train_error : ndarray of shape (n_parameter_values)
        The train error of the ensemble for each parameter value (each ensemble is evaluated only on data that it has seen
        during training.
    member_test_error : ndarray of shape (n_parameter_values, n_test_splits)
        The average test error of the ensemble for each parameter value and test split
    member_train_error : ndarray of shape (n_parameter_values)
        The average train error of the ensemble for each parameter value. Members are evaluated on data that was seen by the
        ensemble during training; there may be examples that were seen by the ensemble but not the individual member

    """

    def __init__(self, parameter_name, parameter_values, loss_func,
                 n_test_splits):
        n_parameter_values = len(parameter_values)
        self.loss_func = loss_func
        self.parameter_name = parameter_name
        self.parameter_values = parameter_values
        self.n_test_splits = n_test_splits
        self.ensemble_risk = np.zeros((n_parameter_values, n_test_splits))
        self.ensemble_bias = np.zeros((n_parameter_values, n_test_splits))
        self.ensemble_variance = np.zeros((n_parameter_values, n_test_splits))
        self.average_bias = np.zeros((n_parameter_values, n_test_splits))
        self.average_variance = np.zeros((n_parameter_values, n_test_splits))
        self.diversity = np.zeros((n_parameter_values, n_test_splits))
        self.test_error = np.zeros((n_parameter_values, n_test_splits))
        self.train_error = np.zeros((n_parameter_values))
        self.member_test_error = np.zeros((n_parameter_values, n_test_splits))
        self.member_train_error = np.zeros((n_parameter_values))


    def update_results(self, decomp, param_idx, errors, split_idx=0, sample_weight=None):
        """
        Function used to update ResultsObject for a new parameter using Decomposition object and list of train/test errors

        Parameters
        ----------
        decomp : Decomposition
            Decomposition object for the experiment
        param_idx : int
            The index of the current parameter in the parameter_values
        errors : list of floats
            List containing (in order):
                Training error averaged over all runs of the experiment
                Test error averaged over all runs of the experiment
                (optional)
                Average member train error
                Average member test error

        Returns
        -------
        None

        """
        self.train_error[param_idx] = errors[0]  # overall error
        self.test_error[param_idx, split_idx] = errors[1][split_idx]  # overall error

        if len(errors) == 4:
            self.member_train_error[param_idx] = errors[2]  # avg member error
            self.member_test_error[param_idx, split_idx] = errors[3][split_idx]  # avg member error

        # We can tell if we have per trial test erorrs by checking if the length of the errors list is odd
        if len(errors) % 2 == 1:
            if not hasattr(self, "per_trial_test_errors"):
                self.per_trial_test_errors = np.zeros((len(self.parameter_values),
                                                       errors[-1].shape[0],
                                                       self.n_test_splits))
            # This also doesn't feel great, is it already filled?
            self.per_trial_test_errors[param_idx, :, split_idx] = errors[-1][:, split_idx]


        self.ensemble_bias[param_idx, split_idx] = np.average(decomp.ensemble_bias,
                                                              weights=sample_weight)

        self.ensemble_variance[param_idx, split_idx] = np.average(decomp.ensemble_variance,
                                                                                  weights=sample_weight)

        self.average_bias[param_idx, split_idx] = np.average(decomp.average_bias,
                                                             weights=sample_weight)

        self.average_variance[param_idx, split_idx] = np.average(decomp.average_variance,
                                                                 weights=sample_weight)

        self.diversity[param_idx, split_idx] = np.average(decomp.diversity, weights=sample_weight)

        self.ensemble_risk[param_idx, split_idx] = np.average(decomp.expected_ensemble_loss,
                                                              weights=sample_weight)
        # logger.debug(f"Update Summary {param_idx},{split_idx}--"
        #              f"ensemble bias: {self.ensemble_bias[param_idx, split_idx]},"
        #              f" ensemble variance: {self.ensemble_variance[param_idx, split_idx]},"
        #              f" average bias: {self.average_bias[param_idx, split_idx]},"
        #              f"average variance: {self.average_variance[param_idx, split_idx]}, "
        #              f"diversity: {self.diversity[param_idx, split_idx]},"
        #              f" ensemble risk{self.ensemble_risk[param_idx, split_idx]},"
        #              f" test error:{self.test_error[param_idx, split_idx]},"
        #              f" train error{self.train_error[param_idx]}")
    def save_results(self, file_path):
            """
            Saves results object to pickle file for later use

            Parameters
            ----------
            file_path : str
                name of file (inlcuding directory) in which results are toe be stored

            Returns
            -------
            None

            """
            with open(file_path, "wb+") as file_:
                pickle.dump(self, file_)

In [11]:
trial_space = np.arange(0,15) / 10
decomp_fn = SquaredLoss
seed = 0
criterion = torch.nn.MSELoss()
epoch_n=7

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


torch_generator = torch.manual_seed(0)

test_dset = trial_dataset(test_data, test_labels, device)


test_loader = torch.utils.data.DataLoader(test_dset, batch_size = 258, shuffle=False)

train_dset = trial_dataset(train_data, train_labels, device)
train_dloader = torch.utils.data.DataLoader(train_dset, batch_size = 128, shuffle=True, generator=torch_generator)
train_unshuffled_dloader = torch.utils.data.DataLoader(train_dset, batch_size = 258, shuffle=False)


In [19]:


# define a trial
def trial_run(trial):

    epoch_results = []
    epoch_train_losses = []

    trial_x, trial_y = trial.get_data

    trial_dset = trial_dataset(trial_x, trial_y, device)
    trial_dloader = torch.utils.data.DataLoader(trial_dset, batch_size = 128, shuffle=True, generator=torch_generator)
    trial_unshuffled_dloader = torch.utils.data.DataLoader(trial_dset, batch_size = 258, shuffle=False)

    # init model
    ensemble = []
    optims = []
    losses = []

    n_estimators = 11
    for member_n in range(n_estimators):
        ensemble.append(SimpleMLP(len(trial_x[0]), 24, 1, torch_generator).to(device))
        optims.append(torch.optim.Adam(ensemble[member_n].parameters()))
        losses.append(None)

    global meta_model 
    meta_model = copy.deepcopy(ensemble[0]).to('meta')
    
    lambda_ = trial.get_param


    for epoch in range(epoch_n):

        trial_results_array = np.zeros((n_estimators, len(test_data)))

        for batch_idx, batch in enumerate(trial_dloader):
            bx, by = batch

            with torch.no_grad():
                ensemble_output, member_output = ens_forward(bx, ensemble)

            member_output = member_output.detach()

            for i, member in enumerate(ensemble):

                optims[i].zero_grad()
                member_pred = member(bx) #predict
                member_grad_output = torch.cat((member_output[:i], member_pred.unsqueeze(dim=0), member_output[i+1:]))
                ens_grad_output = combiner_rule(member_grad_output)

                member_loss = (criterion(member_pred, by.unsqueeze(dim=-1)) - ((lambda_) * criterion(member_pred, ens_grad_output)))
                member_loss.backward()
                optims[i].step()

        with torch.no_grad():
            for i, member in enumerate(ensemble):
                train_preds = member(torch.tensor(trial_x).type(torch.float).to(device))
                epoch_train_losses.append(criterion(train_preds.to(device), torch.tensor(trial_y).unsqueeze(dim=-1).type(torch.float).to(device)).cpu())
                member_preds = member(torch.tensor(test_data).type(torch.float).to(device))
                trial_results_array[i, :] = member_preds.cpu().squeeze()
        
        epoch_results.append(np.array(trial_results_array)) 

    return epoch_results, epoch_train_losses 

# save results
    

study = bvdlib.NCL_Study(trial_space, train_data, train_labels, test_data, test_labels, 
                     num_training, n_trials, decomp_fn, epoch_n)

results = study.run_trials(trial_run)


results.save_results(save_path)

0


KeyboardInterrupt: 